<a href="https://colab.research.google.com/github/dungdt-infopstats/Device-Directed-Speech-Segmentation/blob/main/src_prototype/Phase3_Concat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import json
import pandas as pd

def load_json_folders(root_dir: str) -> pd.DataFrame:
    """
    Duyệt tất cả thư mục con trong root_dir,
    mỗi thư mục chứa một file JSON, load vào DataFrame.

    Parameters
    ----------
    root_dir : str
        Thư mục gốc chứa các thư mục con.

    Returns
    -------
    pd.DataFrame
        DataFrame chứa dữ liệu từ tất cả file JSON.
    """
    data = []
    for subdir in os.listdir(root_dir):
        subpath = os.path.join(root_dir, subdir)
        if os.path.isdir(subpath):
            # tìm file json trong subdir
            for fname in os.listdir(subpath):
                if fname.endswith(".json"):
                    fpath = os.path.join(subpath, fname)
                    with open(fpath, "r", encoding="utf-8") as f:
                        try:
                            obj = json.load(f)
                            data.append(obj)
                        except Exception as e:
                            print(f"⚠️ Lỗi đọc {fpath}: {e}")
    return pd.DataFrame(data)


In [2]:
!gdown 1Tm1UlKNokXDx9cKB3VbAdDMN8kMYcGXX

Downloading...
From (original): https://drive.google.com/uc?id=1Tm1UlKNokXDx9cKB3VbAdDMN8kMYcGXX
From (redirected): https://drive.google.com/uc?id=1Tm1UlKNokXDx9cKB3VbAdDMN8kMYcGXX&confirm=t&uuid=d7083e44-fc58-48b5-9d35-c95761d6f36e
To: /content/all.zip
100% 1.27G/1.27G [00:21<00:00, 58.1MB/s]


In [3]:
!unzip -q all.zip

In [4]:
import os

df_dict = {}
root_path = "all"
for path in os.listdir(root_path):
    folder_path = os.path.join(root_path, path)
    df = load_json_folders(folder_path)
    df_dict[path] = df

# => we have

In [ ]:
!gdown 1ktYfos4wHLUtE1iuC1Pc_wV2cEsSzytm

In [ ]:
!unzip -q sample-5000-trimmed-speech.zip

In [ ]:
!gdown 1bqztSQKwtHABu4_JHFQcRPoxUzAZmEWh

In [ ]:
bad_df = pd.read_csv('sample-5000-bad-list.csv')

In [ ]:
bad_list = set(bad_df['id'].values)

In [ ]:
df_dict['synthesis_command_part_1']

In [9]:
import os
import json
from pydub import AudioSegment
import ast


import os
import json
import ast
from pydub import AudioSegment


class AudioConcatenator:
    def __init__(self, bad_list=None, post_fix=None):
        self.bad_list = set(bad_list) if bad_list else set()
        self.post_fix = post_fix if post_fix else ""
    def _parse_segments(self, type_segments):
        if isinstance(type_segments, str):
            return ast.literal_eval(type_segments)
        if isinstance(type_segments, dict):
            return type_segments
        raise ValueError("Unknown type_segments format")

    def _concat_file(self, row, input_folder, output_folder):
        file_name = f"{row['type']}_{row['id']}"
        type_segments = self._parse_segments(row['type_segments'])

        combined = AudioSegment.silent(duration=0)
        annotations, current_time_ms = [], 0
        num_segments = row['num_segments']

        for i in range(num_segments):
            seg_path = os.path.join(input_folder, file_name, f"{file_name}_seg_{i}{self.post_fix}.wav")
            if not os.path.exists(seg_path):
                print(f"Missing: {seg_path}")
                continue

            segment = AudioSegment.from_wav(seg_path)
            duration_ms = len(segment)

            label = type_segments.get(str(i), "unknown")
            annotations.append({
                "label": label,
                "start": current_time_ms / 1000.0,
                "end": (current_time_ms + duration_ms) / 1000.0
            })

            combined += segment
            current_time_ms += duration_ms

        return self._save_outputs(file_name, combined, annotations, output_folder)

    def _get_non_mix(self, row, input_folder, output_folder):
        file_name = f"{row['type']}_{row['id']}"
        file_path = os.path.join(input_folder, file_name, f"{file_name}{self.post_fix}.wav")

        if not os.path.exists(file_path):
            file_name_full = f"{row['type']}_{row['id']}_full"
            file_path = os.path.join(input_folder, file_name, f"{file_name_full}{self.post_fix}.wav")

        if not os.path.exists(file_path):
            print(f"Missing: {file_path}")
            return None
        segment = AudioSegment.from_wav(file_path)
        duration_ms = len(segment)

        label = "non_active" if row["type"] == "non_active" else "active"
        annotations = [{
            "label": label,
            "start": 0,
            "end": duration_ms / 1000.0
        }]

        return self._save_outputs(file_name, segment, annotations, output_folder)

    def _save_outputs(self, file_name, audio, annotations, output_folder):
        file_output_dir = os.path.join(output_folder, file_name)
        os.makedirs(file_output_dir, exist_ok=True)

        # Save audio
        output_wav = os.path.join(file_output_dir, f"{file_name}_concat.wav")
        audio.export(output_wav, format="wav")

        # Save annotations
        output_json = os.path.join(file_output_dir, f"{file_name}.json")
        with open(output_json, "w", encoding="utf-8") as f:
            json.dump(annotations, f, indent=2, ensure_ascii=False)

        print(f"Saved: {output_wav}, {output_json}")
        return {"wav": output_wav, "json": output_json}

    def process(self, metadata, input_folder, output_folder):
        """Xử lý toàn bộ metadata với input/output folder chỉ định"""
        results = []
        for _, row in metadata.iterrows():
            file_name = f"{row['type']}_{row['id']}"
            file_name_full = f"{row['type']}_{row['id']}_full"
            if file_name in self.bad_list or file_name_full in self.bad_list:
                print(f"Skip {file_name}")
                continue

            if row["num_segments"] != 0:
                result = self._concat_file(row, input_folder, output_folder)
            else:
                result = self._get_non_mix(row, input_folder, output_folder)

            if result:
                results.append(result)

        return results



In [7]:
# config
bad_list = None

In [10]:
from tqdm import tqdm
from pathlib import Path
import os

concat_tool = AudioConcatenator(bad_list=bad_list)
input_root = "all"


results_dict = {}
for folder, df in tqdm(df_dict.items(), desc="Processing folders", unit="folder"):
    name = Path(folder).name
    output_folder = os.path.join("concat_speech", name)
    os.makedirs(output_folder, exist_ok=True)
    input_folder = os.path.join(input_root, name)
    results = concat_tool.process(df, input_folder=input_folder, output_folder=output_folder)
    results_dict[name] = results




Processing folders:   4%|▍         | 2/47 [00:00<00:03, 12.84folder/s]

Saved: concat_speech/synthesis_command_commands_146/single_mix_8694b5db/single_mix_8694b5db_concat.wav, concat_speech/synthesis_command_commands_146/single_mix_8694b5db/single_mix_8694b5db.json
Saved: concat_speech/synthesis_command_commands_146/single_mix_2fbf3053/single_mix_2fbf3053_concat.wav, concat_speech/synthesis_command_commands_146/single_mix_2fbf3053/single_mix_2fbf3053.json
Saved: concat_speech/synthesis_command_commands_146/chain_active_fa4d3fd2/chain_active_fa4d3fd2_concat.wav, concat_speech/synthesis_command_commands_146/chain_active_fa4d3fd2/chain_active_fa4d3fd2.json
Saved: concat_speech/synthesis_command_commands_146/single_active_b22311b8/single_active_b22311b8_concat.wav, concat_speech/synthesis_command_commands_146/single_active_b22311b8/single_active_b22311b8.json
Saved: concat_speech/synthesis_command_commands_146/single_active_1c986873/single_active_1c986873_concat.wav, concat_speech/synthesis_command_commands_146/single_active_1c986873/single_active_1c986873.jso

Processing folders:   9%|▊         | 4/47 [00:00<00:03, 12.48folder/s]

Saved: concat_speech/synthesis_command_commands_147/chain_mix_d06a7908/chain_mix_d06a7908_concat.wav, concat_speech/synthesis_command_commands_147/chain_mix_d06a7908/chain_mix_d06a7908.json
Saved: concat_speech/synthesis_command_commands_147/single_active_418ecaa3/single_active_418ecaa3_concat.wav, concat_speech/synthesis_command_commands_147/single_active_418ecaa3/single_active_418ecaa3.json
Saved: concat_speech/synthesis_command_commands_147/chain_mix_61b1db28/chain_mix_61b1db28_concat.wav, concat_speech/synthesis_command_commands_147/chain_mix_61b1db28/chain_mix_61b1db28.json
Saved: concat_speech/synthesis_command_commands_147/chain_active_c79d3f9f/chain_active_c79d3f9f_concat.wav, concat_speech/synthesis_command_commands_147/chain_active_c79d3f9f/chain_active_c79d3f9f.json
Saved: concat_speech/synthesis_command_commands_147/single_mix_9bd3f40a/single_mix_9bd3f40a_concat.wav, concat_speech/synthesis_command_commands_147/single_mix_9bd3f40a/single_mix_9bd3f40a.json
Saved: concat_spee

Processing folders:  15%|█▍        | 7/47 [00:00<00:05,  7.81folder/s]

...

Saved: concat_speech/synthesis_command_commands_137/single_active_9a151530/single_active_9a151530_concat.wav, concat_speech/synthesis_command_commands_137/single_active_9a151530/single_active_9a151530.json
Saved: concat_speech/synthesis_command_commands_137/single_active_7a21e858/single_active_7a21e858_concat.wav, concat_speech/synthesis_command_commands_137/single_active_7a21e858/single_active_7a21e858.json
Saved: concat_speech/synthesis_command_commands_137/single_mix_91d1d4d0/single_mix_91d1d4d0_concat.wav, concat_speech/synthesis_command_commands_137/single_mix_91d1d4d0/single_mix_91d1d4d0.json
Saved: concat_speech/synthesis_command_commands_137/single_active_c1d3af4b/single_active_c1d3af4b_concat.wav, concat_speech/synthesis_command_commands_137/single_active_c1d3af4b/single_active_c1d3af4b.json
Saved: concat_speech/synthesis_command_commands_137/chain_active_b6cf58cb/chain_active_b6cf58cb_concat.wav, concat_speech/synthesis_command_commands_137/chain_active_b6cf58cb/chain_active_

Processing folders:  96%|█████████▌| 45/47 [00:10<00:00,  6.36folder/s]

Saved: concat_speech/synthesis_command_commands_121/chain_mix_13b6019a/chain_mix_13b6019a_concat.wav, concat_speech/synthesis_command_commands_121/chain_mix_13b6019a/chain_mix_13b6019a.json
Saved: concat_speech/synthesis_command_commands_121/single_active_907a3876/single_active_907a3876_concat.wav, concat_speech/synthesis_command_commands_121/single_active_907a3876/single_active_907a3876.json
Saved: concat_speech/synthesis_command_commands_121/chain_mix_75ff64d5/chain_mix_75ff64d5_concat.wav, concat_speech/synthesis_command_commands_121/chain_mix_75ff64d5/chain_mix_75ff64d5.json
Saved: concat_speech/synthesis_command_commands_121/single_mix_8f971dda/single_mix_8f971dda_concat.wav, concat_speech/synthesis_command_commands_121/single_mix_8f971dda/single_mix_8f971dda.json
Saved: concat_speech/synthesis_command_commands_114/non_active_fc587ebe/non_active_fc587ebe_concat.wav, concat_speech/synthesis_command_commands_114/non_active_fc587ebe/non_active_fc587ebe.json
Saved: concat_speech/synth

Processing folders: 100%|██████████| 47/47 [00:10<00:00,  4.56folder/s]

Saved: concat_speech/synthesis_command_commands_106/single_active_8a5d0a9b/single_active_8a5d0a9b_concat.wav, concat_speech/synthesis_command_commands_106/single_active_8a5d0a9b/single_active_8a5d0a9b.json
Saved: concat_speech/synthesis_command_commands_106/non_active_f8cd9125/non_active_f8cd9125_concat.wav, concat_speech/synthesis_command_commands_106/non_active_f8cd9125/non_active_f8cd9125.json
Saved: concat_speech/synthesis_command_commands_106/chain_active_4de702fc/chain_active_4de702fc_concat.wav, concat_speech/synthesis_command_commands_106/chain_active_4de702fc/chain_active_4de702fc.json
Saved: concat_speech/synthesis_command_commands_106/chain_mix_4432520f/chain_mix_4432520f_concat.wav, concat_speech/synthesis_command_commands_106/chain_mix_4432520f/chain_mix_4432520f.json
Saved: concat_speech/synthesis_command_commands_106/non_active_7e6a8490/non_active_7e6a8490_concat.wav, concat_speech/synthesis_command_commands_106/non_active_7e6a8490/non_active_7e6a8490.json
Saved: concat_

In [11]:
!zip sample-5000-concat.zip concat_speech -r

Streaming output truncated to the last 5000 lines.
  adding: concat_speech/synthesis_command_commands_145/single_mix_3f322c8f/single_mix_3f322c8f_concat.wav (deflated 25%)
  adding: concat_speech/synthesis_command_commands_145/single_mix_1cb8bf5c/ (stored 0%)
  adding: concat_speech/synthesis_command_commands_145/single_mix_1cb8bf5c/single_mix_1cb8bf5c.json (deflated 45%)
  adding: concat_speech/synthesis_command_commands_145/single_mix_1cb8bf5c/single_mix_1cb8bf5c_concat.wav (deflated 26%)
  adding: concat_speech/synthesis_command_commands_145/non_active_d7ec438c/ (stored 0%)
  adding: concat_speech/synthesis_command_commands_145/non_active_d7ec438c/non_active_d7ec438c.json (deflated 18%)
  adding: concat_speech/synthesis_command_commands_145/non_active_d7ec438c/non_active_d7ec438c_concat.wav (deflated 28%)
  adding: concat_speech/synthesis_command_commands_145/non_active_44755183/ (stored 0%)
  adding: concat_speech/synthesis_command_commands_145/non_active_44755183/non_active_447551

In [13]:
!pip install mutagen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 4.5 MB/s eta 0:00:00


In [14]:
import os
from mutagen import File

def total_audio_duration_hours(folder_path: str) -> float:
    """
    Tính tổng thời lượng (giờ) của toàn bộ file âm thanh trong folder (bao gồm thư mục con).
    Dùng mutagen để đọc metadata (nhanh hơn nhiều so với decode audio).

    Args:
        folder_path (str): đường dẫn đến thư mục

    Returns:
        float: tổng số giờ của tất cả file âm thanh
    """
    total_seconds = 0.0

    for root, _, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                audio = File(file_path)
                if audio is not None and audio.info is not None:
                    total_seconds += audio.info.length
            except Exception:
                # bỏ qua file không đọc được
                continue

    return total_seconds / 3600.0


# ví dụ dùng
if __name__ == "__main__":
    folder = "concat_speech"
    hours = total_audio_duration_hours(folder)
    print(hours)


7.023309259259259
